# <span style="color:#E600E6">GROUP SUMMARY STATISTICS</span>
* In the previous session we computed summary statistics across table columns.
* However, this is not enough, as we will often want even more information, which we can get by computing summary statistics for discrete groups in our data.

* We can use Group Summary Statistics to answer questions such as: 
    > * Which major has the least unemployment rate?
    > * Which major pays the best?
    > * What's the total number of Female graduates per major?


We'll continue using the [data behind the story The Economic Guide To Picking A College Major](https://github.com/fivethirtyeight/data/tree/master/college-majors).

In [3]:
%%capture
%load_ext sql
%sql sqlite:///Jobs.db

In [4]:
%%sql
SELECT *
FROM recent_grads
LIMIT 5;

 * sqlite:///Jobs.db
Done.


index,Rank,Major_code,Major,Major_category,Total,Sample_size,Men,Women,ShareWomen,Employed,Full_time,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
0,1,2419,PETROLEUM ENGINEERING,Engineering,2339,36,2057,282,0.120564344,1976,1849,270,1207,37,0.018380527,110000,95000,125000,1534,364,193
1,2,2416,MINING AND MINERAL ENGINEERING,Engineering,756,7,679,77,0.10185185199999999,640,556,170,388,85,0.117241379,75000,55000,90000,350,257,50
2,3,2415,METALLURGICAL ENGINEERING,Engineering,856,3,725,131,0.153037383,648,558,133,340,16,0.024096386,73000,50000,105000,456,176,0
3,4,2417,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,1258,16,1123,135,0.107313196,758,1069,150,692,40,0.050125313,70000,43000,80000,529,102,0
4,5,2405,CHEMICAL ENGINEERING,Engineering,32260,289,21239,11021,0.341630502,25694,23170,5180,16697,1672,0.061097712,65000,50000,75000,18314,4440,972


## <span style="color:#00FF9E">IF/THEN LOGIC in SQL</span>
* We start of with the **IF/THEN** in SQL.
* It's used to create new columns based on some conditions from other columns in the table.
* Example: Say we want to indicate whether a Major has LOW, MEDIUM or HIGH share of women, like in the table below.

| ShareWomen | Group |
| :---: | :---: |
| < 0.2 | Low |
| <= 0.5 | Medium |
| > 0.5 | High |

* We can accomplish this the **CASE** clause in SQL.

In [7]:
%%sql
SELECT
    ShareWomen,
    CASE
        WHEN ShareWomen < 0.2 THEN "LOW"
        WHEN ShareWomen < 0.5 THEN "MEDIUM"
        ELSE "HIGH"
    END AS "ShareWomenGROUP"
FROM recent_grads
LIMIT 10;


 * sqlite:///Jobs.db
Done.


ShareWomen,ShareWomenGROUP
0.120564344,LOW
0.10185185199999999,LOW
0.153037383,LOW
0.107313196,LOW
0.341630502,MEDIUM
0.144966965,LOW
0.535714286,HIGH
0.44135557299999995,MEDIUM
0.139792801,LOW
0.437846874,MEDIUM


The genral Syntax of the CASE clause looks like this.

>  CASE <br>
   &emsp;WHEN <condition_1> THEN <value_1> <br>
   &emsp;WHEN <condition_2> THEN <value_2> <br>
   &emsp;ELSE <value_3> <br>
   END AS <new_column_name>

#### Let's break it down.
> * **CASE** - Indicates beginning of the conditional statement.
> * **WHEN** - Tests the given condition.
> * **THEN** - Returns the given value if the condition was TRUE.
> * **ELSE** - Indicates the fall value incase all conditions above it fail.
> * **END** - Indicates the end of the conditional statement.

### <span style="color:#00FF9E">Exercise</span>
Write a SQL query that displays, with the alias **Sample_category**, the column with values that are a result of the following rules:
* Small if Sample_size is smaller than 200.
* Medium if Sample_size is equal to or higher than 200, and smaller than 1000.
* Large if Sample_size is equal to or higher than 1000.

In [10]:
%%sql
SELECT 
    Major
    Sample_size,
    CASE
        WHEN Sample_size < 200 THEN "Small"
        WHEN Sample_size < 1000 THEN "Medium"
        ELSE "Large"
    END AS Sample_category
FROM recent_grads
LIMIT 5;

 * sqlite:///Jobs.db
Done.


Sample_size,Sample_category
PETROLEUM ENGINEERING,Small
MINING AND MINERAL ENGINEERING,Small
METALLURGICAL ENGINEERING,Small
NAVAL ARCHITECTURE AND MARINE ENGINEERING,Small
CHEMICAL ENGINEERING,Medium


...

## <span style="color:#00FF9E">Calculating GROUPED SUMMARY Statistics </span>
* Grouped statistics are applicable only to columns whose values represent categories.
* So we are going to group the data by each unique category in the column.
* **GROUP BY** clause is used to group rows that have the same values in one or more columns.
* When we use this clause, SQL creates a group for every unique value in the column or set of columns you are grouping by, the aggregates the summary statistics for each group.

* Example: We want to get the total number of unemployed for each Major_category.

In [15]:
%%sql
SELECT 
    Major_category,
    SUM(Unemployed) AS Total_unemployed
FROM recent_grads 
GROUP BY Major_category;

 * sqlite:///Jobs.db
Done.


Major_category,Total_unemployed
Agriculture & Natural Resources,3824
Arts,28228
Biology & Life Science,22854
Business,79877
Communications & Journalism,26852
Computers & Mathematics,18373
Education,24969
Engineering,29817
Health,22213
Humanities & Liberal Arts,51101


* You can as well calculate multiple statistics for each group.

In [25]:
%%sql
SELECT 
    Major_category, 
    SUM(Unemployed) AS Total_unemployed,
    SUM(Total) AS Total_graduates
FROM recent_grads
GROUP BY Major_category;

 * sqlite:///Jobs.db
Done.


Major_category,Total_unemployed,Total_graduates
Agriculture & Natural Resources,3824,79981
Arts,28228,357130
Biology & Life Science,22854,453862
Business,79877,1302376
Communications & Journalism,26852,392601
Computers & Mathematics,18373,299008
Education,24969,559129
Engineering,29817,537583
Health,22213,463230
Humanities & Liberal Arts,51101,713468


* Its also possible to group data by multiple columns:

    > SELECT agg_1, agg_2 <br>
    > FROM table_name <br>
    > GROUP BY col_a, col_b; 

## <span style="color:#00FF9E">Filtering by Aggregate Columns</span>
* Sometimes you want to filter you grouped data by an aggregate operation or column.
Example: Say we group our data by Major_category and we want only the categories whose average **ShareWomen** is greater than 0.7.
* We have to filter our data by the average ShareWomen, in this case we cannot use **WHERE** clause.
* WHERE clause filters data before aggregation, this is because in the order of execution WHERE is executed before GROUP BY.
* To group data by an aggregate, we use the **HAVING** clause.

In [29]:
%%sql
SELECT 
    Major_category,
    AVG(ShareWomen) AS AverageShareWomen
FROM recent_grads
GROUP BY Major_category
HAVING AverageShareWomen > 0.7;

 * sqlite:///Jobs.db
Done.


Major_category,AverageShareWomen
Psychology & Social Work,0.7777631628888888


## <span style="color:#00FF9E">Casting</span>

In [31]:
%%sql
SELECT 
    Major_category, 
    SUM(Unemployed) AS Total_unemployed,
    SUM(Total) AS Total_graduates,
    (SUM(Unemployed) / SUM(Total)) * 100 AS UnemploymentRate
FROM recent_grads
GROUP BY Major_category;

 * sqlite:///Jobs.db
Done.


Major_category,Total_unemployed,Total_graduates,UnemploymentRate
Agriculture & Natural Resources,3824,79981,0
Arts,28228,357130,0
Biology & Life Science,22854,453862,0
Business,79877,1302376,0
Communications & Journalism,26852,392601,0
Computers & Mathematics,18373,299008,0
Education,24969,559129,0
Engineering,29817,537583,0
Health,22213,463230,0
Humanities & Liberal Arts,51101,713468,0


* You see our **UnemploymentRate** column has all zeros.
* The reason for this is the **Total_unemployed** and **Total_graduates** columns are both integers.
* Typically in SQLite and several other SQL dialects, when you divide integers and floats, you get floats.
* But when you divide integers and integers you get integers.
* To get a float in our **UnemploymentRate** column we need to convert on of the two columns to float.
* This is done by **CASTING**.

In [36]:
%%sql
SELECT 
    Major_category, 
    SUM(Unemployed) AS Total_unemployed,
    SUM(Total) AS Total_graduates,
    (CAST(SUM(Unemployed) AS Float) / SUM(Total)) * 100 AS UnemploymentRate
    -- We can round off by nesting the line above in ROUND function
FROM recent_grads
GROUP BY Major_category;

 * sqlite:///Jobs.db
Done.


Major_category,Total_unemployed,Total_graduates,UnemploymentRate
Agriculture & Natural Resources,3824,79981,4.781135519685925
Arts,28228,357130,7.904124548483746
Biology & Life Science,22854,453862,5.035451304581569
Business,79877,1302376,6.133175058508449
Communications & Journalism,26852,392601,6.839513908522902
Computers & Mathematics,18373,299008,6.144651648116438
Education,24969,559129,4.465695751785366
Engineering,29817,537583,5.546492355599042
Health,22213,463230,4.795242104354208
Humanities & Liberal Arts,51101,713468,7.162339446198008


### <span style="color:#00FF9E">Conclusion</span>
In this session we've about
* IF/THEN logic in SQL
* Grouping data with GROUP BY
* Filtering results with HAVING
* TYPE CASTING.

# END